# Simulering av Evolusjon av Antibiotikaresistente Bakterier

En av hovedmålene med dette prosjektet er å reprodusere dette eksperimentet https://www.youtube.com/watch?v=plVk4NVIUh8 
ved Harvard Medical School ved hjelp av en numerisk simulering av bakterier!

I tillegg er målet at det skal hjelpe deg bygge opp en dypere forståelse av hvordan betingelsene
1. Genetisk variasjon
2. Arv av arvestoffet
3. Miljøvariasjon

leder til evolusjon.

## Python biblioteker 

Det er noen Python biblioteker du kommer til å trenge gjennom dette prosjektet.

1. **Numpy** for numerisk Python. Inneholder mange nyttige funksjoner som forenkler kodeskrivingen. Spesielt vil du bruke denne til å trekke tilfeldige tall! For å importere dette, skriver man dette på toppen av koden:
    ```Python
    import numpy as np
    ```
    Spesielt kommer du til å trenge å trekke tilfeldige tall ved hjelp av Numpy. De to tilfellene som er særdeles nyttige er
    1. Tilfeldige *heltall* mellom to heltall `a` og `b`. Dette kan gjøres med
        ```Python
        r = np.random.randint(low=a, high=b) # Den inkludere nederste grense, men ikke øverste. 
        ```  
        Merk at her inkluderes ikke den øvre grensen, så hvis du trenger å inkludere denne, må du skriv `high=b + 1` i stedet.
    2. Tilfeldig *reelle* tall mellom `0` og `1`. Dette kan gjøres med
        ```Python
        r = np.random.uniform(low=0, high=1)  # Reelle tall ("desimaltall") mellom 0 og 1.
        ```
     
2. **Matplotlib** for plotting og animasjoner. Du kommer til å bruke denne både til å lage figurer og animasjoner. For å importere dette skriver man dette på toppen av koden:
    ```Python
    import matplotlib.pyplot as plt
    ```
    Visualisering blir nyttig etter hvert, men det kommer vi tilbake til litt senere.

3. Hvis du ønsker litt **feedback** om hvor langt du er i en for-løkke som bare skal løpe over et bestemt antall iterasjoner `n_iter`, kan du bytte ut `range` med `trange` fra et Python bibliotek som heter `tqdm`. Du må da legge til `from tqdm import trange` (gjerne helt øverst i koden din). En for-løkke kan da skrives som
    ```Python
    for i in trange(n_iter):
        #koden din her
    ```
    Så får du en progresjonsbar som forteller deg hvor langt i simuleringen du er. Nice!

## Bakterie biblioteket

Du får utdelt et bibliotek kalt `Bakterie` som inneholder innebygde egenskaper til en bakterie. Denne ligger i Python filen `bakterie.py`.
For å benytte deg av denne datatypen, må du skrive følgende linje på toppen av koden din:

```Python
from bakterie import Bakterie
```

**Egenskapene** til bakterien er kodet som:

| Egenskap | Variabelnavn | Beskrivelse |
|---|---|---|
| x  | `x`  | x-koordinaten til bakterien.  |
| y  | `y`  | y-koordinaten til bakterien.  |
| Posisjon  | `posisjon`  | (x, y)-koordinaten til bakterien.  |
| Alder | `alder` | Alderen til bakterien. |
| Resistans | `resistans` | Graden av antibiotikaresistans. Er et tall mellom 0 og 1. Hvis 0, har den 0% resistans. Hvis 1, har den 100% resistans. |

# Hvordan lager man og bruker en bakterie?
For å lage en bakterie må vi gi den noen startegenskaper:

- `x_celler`: Antall x-koordinater som finnes i x-retning der bakterien lever.
- `y_celler`: Antall y-koordinater som finnes i y-retning der bakterien lever.
- `x`: x-koordinaten som bakterien starter på. Hvis denne ikke spesifiseres, samples den tilfeldig mellom 0 og `x_celler - 1`
- `y`: y-koordinaten som bakterien starter på. Hvis denne ikke spesifiseres, samples den tilfeldig mellom 0 og `y_celler - 1`
- `resistans`: Graden av antibiotikaresistans bakterien har. Hvis ikke oppgitt, er den satt til 0 som betyr at bakterien ikke er har noen grad av antibiotikaresistans.


Her er et eksempel på hvordan vi lager en bakterie:

In [3]:
from bakterie import Bakterie # Må importere Bakterie for å kunne bruke den 

In [4]:
# Lager en bakterie som lever i et 10 x 10 stort området med startposisjon (5, 4)
bakterie = Bakterie(
    x_celler=10,
    y_celler=10,
    x=5,
    y=4,
    resistans=None, # Resistans settes til 0 hvis den ikke sendes inn eller er satt til `None`.
)

#Printer ut litt info om bakterien som du kan bruke til å sjekke om alt fungerer som det skal
print(bakterie)

Bakterie(
            x = 5
            y = 4
            posisjon = (5, 4)
            alder = 0
            resistans = 0
        )


Vi kan hente ut egenskapene til bakterien som dette:

In [5]:
x = bakterie.x
y = bakterie.y
posisjon = bakterie.posisjon
alder = bakterie.alder
resistans = bakterie.resistans

In [10]:
print(f"x = {x}")
print(f"y = {y}")
print(f"posisjon = {posisjon}")
print(f"alder = {alder}")
print(f"resitans = {resistans}")

x = 0
y = 4
posisjon = (5, 4)
alder = 0
resitans = 0


Vi kan endre egenskapene til bakterien også. Her er noen eksempler:

In [11]:
bakterie.alder += 1 # Øker alderen til bakterien med 1. Samme som å skrive `bakterie.alder = bakterie.alder + 1`
bakterie.x += 1 # Flytter bakterien en celle til høyre. Samme som å skrive `bakterie.x = bakterie.x + 1`
bakterie.y -= 1 # Flytter bakterien en celle ned. Samme som å skrive `bakterie.y = bakterie.y - 1`

Vi kan sjekke hva som skjedde med bakterien sine egenskaper ved å bruke `print(bakterie)`:

In [12]:
print(bakterie)

Bakterie(
            x = 6
            y = 3
            posisjon = (6, 3)
            alder = 1
            resistans = 0
        )


# Modell for bakterien

Å lage en modell for bakterien er nødvendig utover det som er lagt inn i `Bakterie` datatypen. Den lagrer bare data om bakterien som `alder`, `x`- og `y`-koordinat, `posisjon` og `resistans`. Dette er grunnleggende byggesteiner i modellen for bakterien, men det mangler mye her. 

For å lage ferdig en modell for bakterien, er det mange valg vi kan ta. Men det er et par punkter vi bør utfylle for at det skal ligne på en virkelig bakterie:

1. **Bevegelse**. Bakterien må kunne bevege seg rundt i petriskålen.
    - Det er mange måter man kan modellere bevegelsen til bakterier på.
    - Men hvis man ser på en video på Youtube eller kanskje er så heldig å ser på dem gjennom et heftig mikroskop, så ser bevegelsen til bakterier noe tilfeldig ut.
    - Du kan derfor generere bevegelsen til bakteriene **helt tilfeldig**!
2. **Petriskålen**. Petriskålen er på en måte *Universet* til bakteriene i denne simuleringen. 
    - Petriskålen er rektangelformet med `x_celler` posisjoner i x-retning og `y_celler` posisjoner i y-retning. Dette gir `x_celler * y_celler` ruter som en bakterie kan befinne seg på.
    - For eksempel, hvis `x_celler = 5` og `y_celler = 4`, vil det være $4\cdot 5 = 20$ ruter i rektangelet som bakterien kan befinne seg på.
    - Vi kunne modellert dette med flyttall også, men det er litt enklere å gjøre dette med heltall.
3. **Reproduksjon**. Bakteriene må kunne reprodusere/formere seg ved å arve "arvestoffet" stoffet sitt videre til neste generasjon. Bakterier kan ikke reprodusere seg helt fritt i populasjon. Etter hvert som populasjonen vokser, vil konkurranse om ressursene vokse og reproduksjon blir vanskeligere og vanskeligere. En måte å modellere denne prosessen på er å introdusere en sannsynlighet $p(n)$ der $n$ er antall bakterier i populasjonen, og bruke funksjonsuttrykket:
    $$
    p(n) = 2\left(1- \frac{1}{1 + e^{-b\cdot n}}\right)
    $$
    
    - Her er $b$ er en konstant som styrer *bæreevnen* til populasjonen (det vil si den setter en grense på maksimalt antall individer i   populasjonen). Denne kommer til å bli nyttig å kontrollere, siden svært mange bakterier vil gi utrolig treig kode. Treig kode er vanskelig å teste. Generelt sett vil en **liten verdi for $b$ gi en stor populasjon og motsatt**. 
    - Når $n$ er liten er $p(n) \approx 1$ som betyr at det er lett for bakterien og formere seg når det er få individer i populasjonen. Tilsvarende, vil $p(n) \to 0$ når $n \to \infty$ (aka når $n$ blir veldig stor)
    - For øvrig er $e \approx 2.718$ kalt for *Eulers tall* og er innbygd i Numpy som en eksponentialfunksjon $f(x) = e^x$ som i Numpy er `np.exp(x)`.  

4. **En representasjon av arvestoffet for antibiotikaresistans**. Det er fryktelig vanskelig å modellere arvestoffet nøyaktig, selv for en enkel bakterie. I stedet er trikset å modellere fenotypen den arvematerialet koder for. Her er du interessert i å modellere antibiotikaresistans. En enkel måte å modellere dette på er å la antibiotikaresistansen til bakterien være et tall mellom `0` og `1`, der `0` representerer 0% antibiotikaresistans og `1` representerer 100% resistans. Jo høyere grad av resistans bakterien har, jo høyere sannsynlighet har bakterien for å overleve i en miljø med antibiotika.
5. **Mutasjoner**. Genetisk variasjon er en essensiell betingelse for at evolusjon skal finne sted. Siden bakterier i praksis lager en kopi av seg selv når de formerer seg, vil mutasjoner spille en essensiell rolle for å introdusere genetisk variasjon i populasjonen. Det du kan gjøre for å få lage en relativt realistisk modell er å anta at:
    -  Sannsynligheten for at en mutasjon oppstår hos en bakterie er $p = 1 / 10^5 = 10^{-5}$. Det vil si at det skjer ca. 1 mutasjon per 100 000 reproduksjoner. I python kan for øvrig dette skrives som `p = 1e-5`.
    - Anta at hvis arvestoffet skal mutere og avgis til avkommet, at antibiotikaresistansen alltid øker. Men la den øke litt tilfeldig, men aldri la den komme høyere enn akkurat `1`.
    - Vi kommer tilbake til hvordan du kan modellere mutasjonsprosessen med kode lenger ned!

6. **Dødsfall**. Jepp, dystert som det er, så varer ingen liv evig og det gjelder også for bakterier. 
    - I programmet du skriver må du derfor gi bakteriene en endelig levetid og drepe bakterien når dens livsløp passerer den maksimale alderen den kan ha.
    - Kodemessig betyr dette at du må øke `alder` egenskapen til alle bakteriene for hver tidsenhet som går i programmet. 
    - For enkelhets skyld kan du anta at én iterasjon i en for-løkke er ett "bakterieår", også sette en maksimal alder i programmet. Når bakterien passerer alderen, sletter du bakterien fra programmet. Hvis du har en liste med bakterier som heter `bakterier` kan du skrive følgende for å "drepe" en bakterie som er for gammel:

    ```Python
    for bakterie in bakterier:
        if bakterie.alder >= max_alder:
            bakterier.remove(bakterie)
    ``` 

## Modell for Miljøet

Du trenger en modell for petriskålen som bakteriene skal leve i. Vi har allerede nevnt at den kan modelleres som et rektangel bestående av `x_celler * y_celler`
ruter. Men en viktig betingelse for evolusjon er at det er **miljøvariasjon**. 
For evolusjon av antibiotikaresistente bakterier, kan denne miljøvariasjonen være varierende styrke av antibiotika. 
Siden bakterienes antibiotikaresistans kan modelleres med et tall mellom 0 og 1, kan man modellere antibiotikastyrken på samme vis. 
For at det skal ligne på eksperimentet gjort ved Harvard Medical School, kan du lage ulike soner hvor du stadig har et tall som representerer antibiotikaresistansen som er stadig nærmere 1. Lar vi $A(x)$ være antibiotikastyrken, kan du modellere denne som

$$
A(x) = 
\begin{cases}
    0, \quad \text{hvis} \quad 0 \leq x < 0.2 \cdot x_\text{celler} \\
    0.2, \quad \text{hvis} \quad 0.2 \cdot x_\text{celler} \leq x < 0.4 \cdot x_\text{celler} \\
    0.6, \quad \text{hvis} \quad 0.4 \cdot x_\text{celler} \leq x < 0.6 \cdot x_\text{celler} \\
    0.8, \quad \text{hvis} \quad 0.6 \cdot x_\text{celler} \leq x < 0.8 \cdot x_\text{celler} \\
    0.99, \quad \text{hvis} \quad 0.8 \cdot x_\text{celler} \leq x
\end{cases}
$$

Du kan selvfølgelig velge tallene annerledes enn dette også. 
Vi kommer tilbake til hvordan dette kan brukes lenger ned.

# TODO: forslag til delmål for prosjektet

1. Lage en funksjon som flytter bakterien til en ny posisjon.
2. Lage en liste fylt med bakterier og finn ut hvordan man flytter alle sammen.
3. Lag en algoritme som simulerer (utvikler) bakteriepopulasjonen over tid.

## Første steg: **å flytte bakterien**

Siden hver bakterie skal flyttes, og hver slik bakterie kan flyttes etter samme regler, er det lurt å lage en **funksjon** som flytter bakterien. Det er en svært nyttig metode å lage funksjonen på som gjør at koden din blir mer fleksibel, og det er gjennom å lage en funksjon som gir deg en funksjon! Høres komplisert ut, men vil se ut som dette:

```Python
def lag_flytte_funksjon(x_celler, y_celler):
    def flytt(bakterie):
        # TODO skriv din kode som flytter bakterien
    return flytt
```

Tenker vi oss at vi gir petriskålen 20 ruter langs `x`-retning og 10 ruter langs `y`-retning, kan man lage en funksjon som flytter bakteriene med 

```Python
flytt = lag_flytte_funksjon(x_celler=20, y_celler=10) # et miljø med 10 x 10 celler.
```

For å flytte bakterien kan du da skrive 

```Python
bakterie = flytt(bakterie) # Flytter bakterien og returnerer bakterien med dens nye posisjon.
```

**Jobben din er å fylle inn kode der det står # TODO**!

Men hvordan skal du flytte bakterien? Den enkleste løsningen som *garanterer* at bakterien kommer til å kunne nå alle mulige posisjoner i petriskålen over lang nok tid er:
1. Genererer forflytning i `x`-retning og `y`-retning helt tilfeldig! 
2. La endringen være enten `+1` eller `-1` i begge retninger.

*Kodehint*: Du kan hente ut enten `+1` eller `-1` tilfeldig med
```Python
import numpy as np # Pass på å importere dette på toppen av koden din! 

# Trekke tilfeldig endring i x-retning:
dx = np.random.choice([-1, 1]) #Setter endringen `dx` til enten `+1` eller `-1` med 50% sjanse for hver.
```

Du må passe på at bakterien ikke beveger seg utenfor området (petriskålen) som har størrelse `x_celler` i `x`-retning og `y_celler` i `y`-retning.

In [1]:
# Skjelettkode for flyttefunksjonen
import numpy as np
def flytte_funksjon(x_celler, y_celler):
    def flytt(bakterie):
        dx = np.random.choice([-1, 1])
        dy = np.random.choice([-1, 1])
        
        if bakterie.x + dx >= 0:
            bakterie.x += dx            
        elif bakterie.x + dx < bakterie.x_celler:
            bakterie.x += dx
        
        if (bakterie.y + dy >= 0):
            bakterie.y += dy
        elif (bakterie.y + dy < bakterie.y_celler):
            bakterie.y += dy
        
        return bakterie
    return flytt

## Andre steg: **Lage en populasjon med bakterier**

Du er nødt å fylle opp en populasjon med bakterier. Dette i seg selv er ikke rett frem å finne ut av hvordan man gjør ut å lært 
seg hvor **lister** i Python funker. Det er en **beholder** du kan fylle opp med vilkårlige objekter i Python. Noen eksempler er:

```Python
heltalls_liste = [0, 1, 2, 3, 4, 5]
mikset_liste = [0, "lol", None, True, 1e-3] # Listen inneholder objekter med datatypene int, str, None, bool og float 
```

Man har en genial måte å lage lister på som kalles for **list-comprehension** eller bare **listcomp** (har ikke noen norsk betegnelse så vidt jeg vet).
Hvis du ønsker å fylle opp en liste med bakterier, kan det gjøres slik:

```Python
x_celler = 20
y_celler = 10
n_bakterier = 100 # Antall bakterier
bakterier = [Bakterie(x_celler=x_celler, y_celler=y_celler, x=None, y=None, resistans=None) for _ in range(n_bakterier)]
```

Her genereres `x` og `y` tilfeldig slik at `x` ligger mellom `0` og `x_celler`, og `y` ligger mellom `0` og `y_celler`. I simuleringen du skal skrive, må du nok begrense hvor i petriskålen startverdiene til `x` og `y` er, men her ser du en måte du kan fylle opp en liste med bakterier på. 

En mer typisk løsning som gjør akkurat det samme, men som krever flere linjer med kode ville vært:

```Python
x_celler = 20
y_celler = 10
n_bakterier = 100 # Antall bakterier
bakterier = [] # Lag en tom liste som skal fylles opp
for _ in range(n_bakterier):
    bakterier.append(
        Bakterie(
            x_celler=x_celler,
            y_celler=y_celler,
            x=None,
            y=None,
            resistans=None,
        )
    )
```

Noen ganger er dette lettere å lese, men det viser seg å ofte gi *tregere* kode!

*For øvrig! Hvis du lurer på hvorfor det er brukt `_` i for løkka i stedet for en variabel `i`, så skyldes det at hvis du ikke bruker variabelen du itererer over i for-løkka, bør den settes til `_` for å indikere for leseren at løkke-variabelen ikke brukes til noe i for-løkka.*

## Tredje steg: Lage en algoritme som utvikler bakteriene over tid

Her overlater jeg deg til å tenke litt mer på egenhånd siden du nå har ganske mange av byggesteinene du trenger for å skrive en tidsutvikling av bakteriepopulasjonen. 
Du skal likevel få et par flere hint du kan benytte deg av her. 

1. For å bestemme om en bakterie kan reprodusere eller ikke, må du ha et par ingredienser til stedet. 
    - Sannsynlighetsfunksjonen $p(n)$ som vi introduserte i stad må lages. Igjen lønner det seg å gjøre som proffene å lage en funksjon som lager en funksjon. Her er en skjelettkode for dette:
        ```Python
        def lag_sannsynlighetsfunksjon(b):
            def p(n):
                b = 0.001
                sannsynlighet = 2 * (1 - 1/(1 + np.exp(-b * n)))
                return sannsynlighet
            return p

        # Eksempel på hvordan du lager en sannsynlighetsfunksjon p(n) for en bestemt verdi av `b`
        p = lag_sannsynlighetsfunksjon(b=0.001) # Lage en funksjon for p(n) der b er satt til b=0.001. 
        ```
    - For å sjekke om bakterien skal reprodusere, kan du gjøre det som kalles for **slice sampling**, som betyr:
        - Trekk et tilfeldig tall $r$ mellom 0 og 1. 
        - Hvis $p(n) > r$, så får bakterien reprodusere, hvis ikke får ikke bakterien reprodusere.
        - For å hente ut hvor mange bakterier du har i populasjonen, kan du skrive `n = len(bakterier)` i Python. Funksjonen `len` henter ut lengden på en liste,
        som betyr antall elementer i listen. Siden `bakterier` er en liste med bakterier og hvert element er en bakterie, vil `len(bakterier)` gi antall bakterier i lista (eller da i populasjonen din)! 
2. For å bestemme om bakterien som blir født når en gammel bakterie formerer seg skal få et mutert gen, kan du også gjøre slice sampling! Hvis du har satt `mutasjons_sannsynlighet = 1e-5`, vil du bestemme om en bakterie skal få et mutert arvestoff eller ikke med
    ```Python
        r = np.random.uniform(low=0, high=1)
        if mutasjons_sannsynlighet > r:
            # Lag en bakterie med som har mutert arvestoffet og derfor får økt antibiotikaresistans her
        else:
            # Lag en bakterie med akkurat samme resistans som "forelder"-bakterien. 
    ```

3. For å bestemme om bakterien skal dø eller ikke av antibiotikaen i miljøet den lever i, kan du benytte deg av funksjonen for antibiotikastyrken $A(x)$, som ble definert under modellen for petriskålen, på følgende måte:
    - Finn `x` - posisjonen til bakterien og regn ut antibiotikastyrken i området bakterien befinner seg i.
    - Hent ut antibiotikaresistansen $R$ til bakterien.
    - Trekk et tilfeldig tall $r$ mellom 0 og 1.
    - Hvis $R - A(x) < r$, så overlever bakterien iterasjonen.


Jobben din er å lage en algoritme du skal skrive i Python, men et par ting å tenke på er at per iterasjon (per "bakterieår") bør du ha med:
1. Alle bakteriene flytter seg ett steg.
2. Alle bakteriene "forsøker" å formere seg etter regelen introdusert over.
3. Alle bakterier som er for gamle drepes.

Så reptererer du dette mange ganger! 

In [ ]:
import bakterie 
n=100
b = 0.001
funksjon = sannsynlighetsfunksjon(b)
resultat = funksjon(n)
print(resultat)

In [49]:
from bakterie import Bakterie
from bakterie import flytte_funksjon

# Lager en bakterie som lever i et 10 x 10 stort området med startposisjon (5, 4)
gammel_bakterie = Bakterie(
    x_celler=10,
    y_celler=10,
    x=5,
    y=4,
    resistans=None, # Resistans settes til 0 hvis den ikke sendes inn eller er satt til `None`.
)

#Printer ut litt info om bakterien som du kan bruke til å sjekke om alt fungerer som det skal
print(gammel_bakterie)

#rar_python = flytte_funksjon(100)
#oppdatert_bakterie = rar_pythonflytt(gammel_bakterie)

Bakterie(
            x = 5
            y = 4
            posisjon = (5, 4)
            alder = 0
            resistans = 0
        )


In [8]:
from bakterie import Bakterie
import numpy as np # Nødvendig pakke
import matplotlib.pyplot as plt # Nødvendig pakkee
import matplotlib.animation as animation
from matplotlib.animation import PillowWriter
from tqdm import trange

print("Hello 1")

#flytting av bakterien
def flytte_funksjon(x_celler, y_celler):
    def flytt(bakterie):
        dx = np.random.choice([-1, 1])
        dy = np.random.choice([-1, 1])

        ny_x_verdi = bakterie.x + dx
        if ny_x_verdi > 0 and ny_x_verdi < x_celler-1:
            bakterie.x = ny_x_verdi       

        ny_y_verdi = bakterie.y + dy
        if ny_y_verdi > 0 and ny_y_verdi < y_celler-1:
            bakterie.y = ny_y_verdi
        

        return bakterie
    return flytt


#Reproduksjon av bakterier
def sannsynlighetsfunksjon(b):
    def p(n):
        sannsynlighet = 2 * (1 - 1/(1 + np.exp(-b * n)))
        return sannsynlighet
    return p

def reproduksjon(mammabakterie):
    antallbakterier = len(bakterier)
    sannsynlighetsbanan = sannsynlighetsfunksjon(0.001)
    sannsynlighet = sannsynlighetsbanan(antallbakterier) 
    r = np.random.uniform(low=0, high=1)
    if r < sannsynlighet:
        return True
    else: return False

#milø for bakterier
x_celler = 100
y_celler = 100
bakterier=[]

#Spawning av bakterier
for teller in range(1): 
    bakterie = Bakterie(
        x_celler= x_celler,
        y_celler= y_celler,
        x=teller,
        y=1, #np.random.randint(y_celler/4),
        resistans=None)
    
    bakterier.append(bakterie)

flytt = flytte_funksjon(x_celler, y_celler)

fig, ax = plt.subplots()    

#simulering og visualisering
imgs = []
for n in trange(100):
    img = np.zeros(shape=(x_celler, y_celler))

    for bakterie in bakterier:
        if reproduksjon(bakterie):
            bakterie = Bakterie(
            x_celler= x_celler,
            y_celler= y_celler,
            x=bakterie.x,
            y=bakterie.y, 
            resistans=None)
    
            bakterier.append(bakterie)
            
        flytt(bakterie)
        img[bakterie.x, bakterie.y] = 1
    #print(img)
    #if n == 0:
        #imgs.append([ax.imshow(img.T, cmap="gray")])
    #else:
    imgs.append([ax.imshow(img.T, animated=True)])

ani = animation.ArtistAnimation(fig, imgs, interval=1000, blit=True)

writer = PillowWriter(fps=2)
print("Skriver banan.gif")
ani.save("banana.gif", writer=writer)
plt.close()

Hello 1


100%|██████████| 100/100 [00:22<00:00,  4.36it/s]


Skriver banan.gif
